# Part 2 - Coherent Detection

In this section of the practical work we will analyse several aspects of optical communication systems using coherent detection, as schematized in the figure below.

<img src="System.png" width=700 height=700 style="float: center"/>

For this part of the work we will need the QAMpy package. This package can be found on https://github.com/ChalmersPhotonicsLab/QAMpy [1]. 
Please install the package. The installation instructions for Windows are available in http://qampy.org/installation/windows.html, and for Linux in http://qampy.org/installation/linux.html. If you are having any problem with the package installation, please tell us.

At the transmitter we will study a binary data source. This data will be used to simulate the preparation needed before transmission, as for example, the modulation formats, the pulse shaping, and the transmitter's response. Then, the transmission of the prepared signal will be studied. Here, several impairments will be considered, as for example, phase noise, dispersion,  Polarization Mode Dispersion (PMD), and  Carrier Frequency Offset (CFO). In each of these steps the signal can be observed in the quadrature domain, in the time domain, and in the frequency domain. At the receiver we will calculate the Symbol Error Rate (SER), the BER, the  Error Vector Magnitude (EVM), and the Mutual Information (MI), and we will apply equalization and analyse signals with pilots. Despite dividing the work of the present section in the three mentioned sub-sections, remark that we cannot study most of the mentioned elements without simulating the preparation, transmission, and measurement of the signals. 

The jupyter notebook file attached on coherent detection contains all code blocks mentioned below numbered by order of appearing.

Importing packages needed:

In [ ]:
from qampy.core import io
from qampy import signals,impairments,helpers,equalisation,io,filtering,phaserec,helpers,theory
from qampy.core import impairments as impair
import numpy as np
from bokeh.io import output_notebook, push_notebook
from bokeh.plotting import figure, show
from bokeh.layouts import row, column, gridplot
import bokeh.palettes as palettes
from bokeh.models import Range1d
import warnings
from plots_def import *
warnings.filterwarnings('ignore')
output_notebook()

## 1. Transmitter

Before we can transmit some data or information between two entities, the data to be exchanged has to be encoded in a way that is suitable for transmission. This can be done, for example, using analog modulation and digital modulation. Here we are going to focus on digital modulation, where a digital sequence is mapped into a set of symbols. Usually this digital sequence is binary, i.e., it only contains 0's and 1's.

### 1.1. Data Source

Here the data source of the information transmitted in our optical communication system will be a  Pseudo-Random Binary Sequence (PRBS). 
To generate a PRBS sequence we will use the function signals.RandomBits(N,seed = seed) available in the QAMpy package. 'N' is the total number of bits in the sequence, i.e., its length, and 'seed' is the seed of the PRBS. 

Use ***Code 2.1*** to generate a PRBS sequence with $2^{6}$ bits using your mecanographic number as the seed.

***Code 2.1:***

In [ ]:
N = ? # number of bits
seed = [?] # mecanographic number

prbs = signals.RandomBits(N,seed=seed)

print('Generated PRBS: ')
print(prbs)

### 1.2. Modulation Formats

Now, we will apply the modulation format to a pseudo-random binary sequence to prepare it for transmission.
Different modulation schemes can be used. We are going to focus on single-carrier modulation formats, where data is transmitted using only one carrier, for example, phase, amplitude, or frequency.
The two modulation formats that will be considered here are the M-symbol Phase Shift Keying (M-PSK) and the M-symbol Quadrature Amplitude Modulation (M-QAM), where $M$ is the number of points of the constellation. Usually, M is equal to $2^{k}$, where $k$ is the number of bits per symbol. Here the binary sequence associated to each point of the modulation formats considered will be given by the gray coding.

We will start by considering the M-PSK scheme. To do so we will use the function  signals.SignalPSKGrayCoded(M,N,nmodes,fb) from QAMpy, where 'M' is the number of points of the M-PSK constellation, 'N' is the number of symbols, 'nmodes' is the number of modes, and 'fb' is the symbol rate of the communication over the data channel. 

2.1. <font color='darkblue'>Define the concept of modes in an optical signal. Give an example of modes in optical signals.</font>

***Code 2.2*** generates a one mode (one polarization) QPSK signal with a total of $2^{5}$ symbols at a symbol rate of 25 GHz. A QPSK signal corresponds to a 4-PSK and 4-QAM modulation, thus having only 4 different points in the constellation. Additionally, the code below represents the real component of the generated QPSK signal.
Here, and for the rest of the work, consider the same seed as considered before, i.e., your mecanographic number. Remark that doing seed = [seed] yields a different result than doing seed = seed.

2.2. <font color='darkblue'>What are the phases of the points of the QPSK modulation format considered in ***Code 2.2***.</font>

2.3. <font color='darkblue'>What is gray mapping?</font>

***Code 2.2:***

In [ ]:
M = 4 
N = 2**5 
fb = 25e9
seed = [?]

sigQPSK = signals.SignalPSKGrayCoded(M, N, nmodes=1, fb=fb, bitclass=signals.RandomBits, seed = seed)

fig1 = figure(output_backend="webgl",plot_height=200, plot_width=700)
fig1.step(np.arange(sigQPSK.shape[1])*1/fb,sigQPSK.real[0,:], line_width=2, alpha=0.7, mode="after", legend_label = 'QPSK signal')
for i in np.arange(sigQPSK.shape[1]):
    fig1.line([i*1/fb,i*1/fb], [min(sigQPSK.real[0,:]), max(sigQPSK.real[0,:])], color='black',dash = 'dashed', legend_label = 'Symbol boundaries')
fig1.xaxis.axis_label = "Time (s)"
fig1.yaxis.axis_label = "Real component"
show(fig1)

***Code 2.2*** was adapted to consider several values of M, considering now $2^{16}$ points, and considering both M-PSK and M-QAM constellations. To study the M-QAM modulation format, we will use the function signals.SignalQAMGrayCoded(M,N,nmodes,fb), similar to the one used in ***Code 2.2*** for M-PSK.
***Code 2.3*** plots the various constellations obtained. 
The plots are obtained using function plot_constellation(signal,legend), where 'signal' is a list of all the signals to plot, and 'legend' is a list of the legend for each signal to be plotted.
Remark that obtaining the QPSK constellation considering 4-QAM instead of 4-PSK yields a different gray mapping.

Different possibilities for M are 4, 8, 16, 32, 64, 128, 256, 512, 1024, 2048,... .
***Code 2.3*** considers M between 4 and 64. Change the values in M for better visualization.

- <font color='darkblue'>Define and distinguish M-QAM and M-PSK modulation formats. Can you observe differences between M-QAM constellation with different values of M?</font>

- <font color='darkblue'>The optimum capacity of an AWGN Channel is obtained for an optimum constellation following a Gaussian distribution. Why are discrete modulation formats used in practice instead of the optimum modulation format? Which constellations, for both modulation formats studied above, do you expect to approximate more the optimum capacity, per unit of bandwidth, of an AWGN channel? The capacity per unit of bandwidth corresponds to the spectral efficiency. For both modulation formats, what is the maximum amount of bits per second per Hertz that each constellation can transmit? (See section 2.4 of [4])</font>

***Code 2.3:***

In [ ]:
M = [4, 8, 16, 32, 64] # Modulation order: number of points in the constellation
N = 2**16 # number of symbols per mode
fb = 25e9
seed = [?]

sigPSK = [signals.SignalPSKGrayCoded(m, N, nmodes=1, fb=fb, bitclass=signals.RandomBits, seed = seed) for m in M]
legend = [str(m)+"-PSK Constellation" for m in M]  
plot_constellation(list(reversed(sigPSK)),list(reversed(legend)))

sigQAM = [signals.SignalQAMGrayCoded(m, N, nmodes=1, fb=25e9, bitclass=signals.RandomBits, seed = seed) for m in M]
legend = [str(m)+"-QAM Constellation" for m in M]  
plot_constellation(sigQAM,legend)

### 1.3. Pulse Shaping

In ***Code 2.4*** we applied a rectangular pulse to the M-PSK constellation, and plotted the signal, the signal's real component and the signal's imaginary component. We consider the sub-function from_bit_array of signals.SignalQAMGrayCoded and signals.SignalPSKGrayCoded. This sub-function has as input the PRBS that we generate using signals.RandomBits. In ***Code 2.5***, the RRCos filter was aplyed to the M-PSK constellation. To consider M-QAM constellations instead of M-PSK constellations, change the function signals.SignalPSKGrayCoded to signals.SignalQAMGrayCoded. Remark that, the titles of the plots must also be changed. 
The signal obtained in ***Code 2.4*** and in ***Code 2.5*** considers a sampling rate ('fs') 4 times the symbol rate ('fb') of the signal. The symbol rate corresponds to the number of bits in one symbol, while the sampling rate is the rate at which the receiver reads the samples, i.e., it is the average number of samples obtained in one second. 

- <font color='darkblue'>Both for the rectangular pulse (***Code 2.4***) and for the RRCos filter (***Code 2.5***), change the value of M of the M-PSK constellation. Consider also M-QAM constellations. For the RRCos filter (***Code 2.5***), change also the roll-off factor ('beta', with 'beta' in the interval ]0,1]). Additionally, for both pulse shapes consider fs equal to fb, 2fb, and 4fb. Comment your observations. Remark that the real and imaginary components of the signal were potted using a step function.</font>

***Code 2.4:***

In [ ]:
M = 4
seed = [?] # mecanographic number
fs = fb*4

fb = 25e9
Ts = 1/fs
Tsymbol = 1/fb

N = int(20*np.log2(M)) # number of bits
prbs = signals.RandomBits(N,seed = seed)

numSymbols = N/np.log2(M)

fig = figure(title='Binary sequence for the '+str(M)+'-QAM Signal',output_backend="webgl",plot_height=200, plot_width=700)
fig.step(np.arange(N)*Tsymbol/np.log2(M),np.reshape(prbs,(-1,)), line_width=2, alpha=0.7, mode="after")
for i in np.arange(numSymbols):
    fig.line([i*Tsymbol,i*Tsymbol], [-0.1, 1.1], color='black',dash = 'dashed')
fig.xaxis.axis_label = "Time (s)"
show(fig)

sigQAM = np.repeat(signals.SignalQAMGrayCoded.from_bit_array(prbs, M, fb=fb),fs/fb)
t = np.arange(sigQAM.shape[0])*Ts
Tsymbol = sigQAM.shape[0]/numSymbols*Ts

fig1 = figure(title='Real component of the '+str(M)+'-QAM Signal',output_backend="webgl",plot_height=200, plot_width=350)
fig1.step(t,sigQAM.real, line_width=2, alpha=0.7)
for i in np.arange(numSymbols):
    fig1.line([i*Tsymbol,i*Tsymbol], [min(sigQAM.real), max(sigQAM.real)], color='black',dash = 'dashed')
fig1.xaxis.axis_label = "Time (s)"

fig2= figure(title='Imaginary component of the '+str(M)+'-QAM Signal',output_backend="webgl",plot_height=200, plot_width=350)
fig2.step(t,sigQAM.imag, line_width=2, alpha=0.7)
for i in np.arange(numSymbols):
    fig2.line([i*Tsymbol,i*Tsymbol], [min(sigQAM.imag), max(sigQAM.imag)], color='black',dash = 'dashed')
fig2.xaxis.axis_label = "Time (s)"

show(row(fig1,fig2))

norm = np.repeat(np.abs(sigQAM),20)
phase = np.repeat(np.angle(sigQAM),20)

t = np.arange(20*sigQAM.shape[0])*Ts/20

fig3 = figure(title=str(M)+'-QAM Signal',output_backend="webgl",plot_height=250, plot_width=700)
fig3.line(t,norm*np.cos(t*2*np.pi*fs + phase), line_width=2, alpha=0.7)
for i in np.arange(numSymbols):
    fig3.line([i*Tsymbol,i*Tsymbol], [min(norm*np.cos(t*2*np.pi*fs + phase)), max(norm*np.cos(t*2*np.pi*fs + phase))], color='black',dash = 'dashed')
fig2.xaxis.axis_label = "Time (s)"
show(fig3)

***Code 2.5:***

In [ ]:
M = 4
beta = 0.01 # roll of factor
seed = [?] # mecanographic number
fs = fb*4

fb = 25e9
Ts = 1/fs
Tsymbol = 1/fb

N = int(20*np.log2(M)) # number of bits
prbs = signals.RandomBits(N,seed = seed)

numSymbols = N/np.log2(M)

fig = figure(title='Binary sequence for the '+str(M)+'-QAM Signal',output_backend="webgl",plot_height=200, plot_width=700)
fig.step(np.arange(N)*Tsymbol/np.log2(M),np.reshape(prbs,(-1,)), line_width=2, alpha=0.7, mode="after")
for i in np.arange(numSymbols):
    fig.line([i*Tsymbol,i*Tsymbol], [-0.1, 1.1], color='black',dash = 'dashed')
fig.xaxis.axis_label = "Time (s)"
show(fig)

sigQAM = signals.SignalQAMGrayCoded.from_bit_array(prbs, M, fb=fb)
sigQAM = sigQAM.resample(fs, Ts = Tsymbol, beta = beta, taps=4001, renormalise=True)
t = np.arange(sigQAM.shape[1])*Ts
Tsymbol = sigQAM.shape[1]/numSymbols*Ts

fig1 = figure(title='Real component of the '+str(M)+'-QAM Signal',output_backend="webgl",plot_height=200, plot_width=350)
fig1.step(t,sigQAM.real[0,:], line_width=2, alpha=0.7)
for i in np.arange(numSymbols):
    fig1.line([i*Tsymbol,i*Tsymbol], [min(sigQAM.real), max(sigQAM.real)], color='black',dash = 'dashed')
fig1.xaxis.axis_label = "Time (s)"

fig2= figure(title='Imaginary component of the '+str(M)+'-QAM Signal',output_backend="webgl",plot_height=200, plot_width=350)
fig2.step(t,sigQAM.imag[0,:], line_width=2, alpha=0.7)
for i in np.arange(numSymbols):
    fig2.line([i*Tsymbol,i*Tsymbol], [min(sigQAM.imag), max(sigQAM.imag)], color='black',dash = 'dashed')
fig2.xaxis.axis_label = "Time (s)"

show(row(fig1,fig2))

norm = np.repeat(np.abs(sigQAM),20)
phase = np.repeat(np.angle(sigQAM),20)

t = np.arange(20*sigQAM.shape[1])*Ts/20

fig3 = figure(title=str(M)+'-QAM Signal',output_backend="webgl",plot_height=250, plot_width=700)
fig3.line(t,norm*np.cos(t*2*np.pi*fs + phase), line_width=2, alpha=0.7)
for i in np.arange(numSymbols):
    fig3.line([i*Tsymbol,i*Tsymbol], [min(norm*np.cos(t*2*np.pi*fs + phase)), max(norm*np.cos(t*2*np.pi*fs + phase))], color='black',dash = 'dashed')
fig2.xaxis.axis_label = "Time (s)"
show(fig3)

- <font color='darkblue'>In ***Code 2.6***, the original signal is re-sampled to have as sampling rate twice the symbol rate, and 0.01 as the filter's roll-off factor. Plot only each second sample of the signal starting from 0, and starting from 1. Explain the differences that you observe between the plots, and compare with the initial signal.</font>

In ***Code 2.6***, and others, the functions plot_histogram(signal,title), plot_power_time(signals,title,L,mode), plot_power_signal_time(signals,title,L,mode), and plot_power_frequency(signals,title), are considered. Here 'signal' is only one signal, while 'signals' is a list of 'signals', 'title' is the title of the plot, 'L' is the number of signal's periods to plot, and 'mode' is the mode index to be plotted, if not all modes are to be plotted.

***Code 2.6:***

In [ ]:
M = 64
seed = [?] # mecanographic number
nmodes = 2
fb = 25e9
N = 2**16
Tsymbol = 1/fb

fs = fb*2
beta = 0.01

sig = signals.SignalQAMGrayCoded(M, N, nmodes=nmodes, fb=fb, bitclass=signals.RandomBits, seed = seed)
sigRes = sig.resample(fs, Ts = Tsymbol, beta = beta, taps=4001, renormalise=True)

plot_histogram(sigRes[:,0::1],'All samples',mode = 0)

### 1.4. Realistic Transmitter Response

Now, we will simulate a realistic transmitter. But first we need to generate the signal to transmit, which is done using ***Code 2.7***.

At this transmitter we will first consider the Digital to Analog Converter (DAC) response, after, the signal is amplified, and lastly we will consider the modulator's response. All these processes, in practice, come with some impairments that we will study in this section.
Have in mind that it may be usefull to consider QPSK instead of 64-QAM to analyse the effects below.

***Code 2.7:***

In [ ]:
M = 64
seed = [?] # mecanographic number
nmodes = 2
fb = 25e9
N = 2**16
fs = fb*2
Ts = 1/fs
Tsymbol = 1/fb
beta = 0.01

sig = signals.SignalQAMGrayCoded(M, N, nmodes=nmodes, fb=fb, bitclass=signals.RandomBits, seed = seed)
sigRes = sig.resample(fs, Ts = Tsymbol, beta = beta, taps=4001, renormalise=True)

#### DAC response

The DACs will transform a digital signal to an analog signal in order to modulate the laser diode output. However, the responce of the DACs depends on the sampling clock, which in practical systems can present jitter (which is when, a presumably periodic signal, presents a deviation from true periodicity). The jitter causes the sampling time for a given digital input to not be exact. Given that the clock jitter is random, the sampling time of the DAC is unpredictable, and the analog output signal will show an amplitude shift.

Using ***Code 2.8***, start by applying different values of quantization, i.e. changing the number of bits in the quantizer, by changing the variable 'quant_bits'. Change also the Effective Number of Bits (ENOB) for the DAC, 'enob', which describes the effective resolution of the system in bits. This slices the signal into discrete levels. You can also change the cutoff frequency of the 2nd order Bessel that simulates the frequency response of the DAC filter used.  Finally, change the ratio of signal left after clipping. Clipping can occur when a signal is transformed by a DAC or an Analog to Digital Converter (ADC) and the range of data at the input of the device is not appropriate for the measurement specifications of that device, leading to a cutting of part of the input signal.

- <font color='darkblue'>Briefly comment the effect of changing the above mentioned parameters: quantization, ENOB, cutoff frequency and clipping ratio.</font>

***Code 2.8:***

In [ ]:
quant_bits = 0 
enob = 6
clip_rat = 1 # betweem 0 and 1
tgt_v = 1
cutoff = fs/2-1

mysig_DAC = sigRes.recreate_from_np_array(impair.sim_DAC_response(sigRes,
    sigRes.fs,enob,clip_rat,quant_bits,cutoff=cutoff))
plot_histogram(mysig_DAC,'quant_bits = '+str(quant_bits),mode = 0)
plot_power_signal_time([sigRes, mysig_DAC],['Original','New: quant_bits = '+str(quant_bits)],L = 100,mode = 0)
plot_power_frequency([sigRes, mysig_DAC],['Original','New: quant_bits = '+str(quant_bits)],mode = 0)

#### Modulator response

A modulator is responsible for modulating the carrier of the signal. The modulator has as input signals, the optical signal to modulate, an electrical signal according to which the modulation will be done, and a bias voltage. 

- <font color='darkblue'>Change, in ***Code 2.9***, the DC bias ('dcsig'), the split imbalance and path dependent loss ('gfactr'), and the chirp factor ('cfactr'). Remark that an ideal Mach-Zehnder Modulator (MZM) with infinite extinction ratio has split imbalance and path dependent loss equal to 1. The chirp is caused by the asymmetry in the electrode design of the MZM, being null for an ideal MZM. Here, the input laser power is assumed to be 0 dBm. Briefly comment the resuts obtained.</font>

***Code 2.9:***

In [ ]:
dcbias=1
gfactr=1
cfactr=0
dcbias_out=0.5
gfactr_out=1

mysig_MR = impair.modulator_response(sigRes,dcbias, gfactr, cfactr, dcbias_out, gfactr_out)  
plot_histogram(mysig_MR,'dcbias ='+str(dcbias),mode = 0)
plot_power_signal_time([sigRes, mysig_MR],['Original','New: dcbias = '+str(dcbias)],L = 100,mode = 0)
plot_power_frequency([sigRes, mysig_MR],['Original','New: dcbias = '+str(dcbias)],mode = 0)

## 2. Transmission

After the signal is prepared we can transmit it through the optical channel. When we have an optical fiber as channel we will have some effects that cause imperfections to the signal. Some of the effects to consider are the phase-noise and \gls{PMD}. Once again, have in mind that it may be usefull to consider QPSK instead of 64-QAM to analyse the effects below.

### 2.1. Phase Noise

The phase noise depends both on the linewidth of the local oscillators and on the signal's sampling frequency, being here represented by a Wiener noise process with a variance given by $\sigma^2=2\pi \frac{\text{lwdth}}{\text{fs}}$, where 'lwdth' corresponds to the combined linewidth of the local oscillators in the system. 

- <font color='darkblue'>In ***Code 2.10***, change 'lwdth' and the sampling rate of the signal. Comment on its effect. Consider, for example, 'lwdth' between $10^1$ and $10^{10}$.</font>

***Code 2.10:***

In [ ]:
fs = fb*2
sigRes_pn = sig.resample(fs, Ts = Tsymbol, beta = beta, taps=4001, renormalise=True)

lwdth = 10**3

mysig_PN = impairments.simulate_transmission(sigRes,lwdth=lwdth)
plot_histogram(mysig_PN,'lwdt = '+str(lwdth),mode = 0)
plot_power_time([sigRes_pn, mysig_PN],['Original','New: ' + 'lwdt = '+str(lwdth)],L = 50,mode = 0)
plot_power_frequency([sigRes_pn, mysig_PN],['Original','New: ' + 'lwdt = '+str(lwdth)],mode = 0)

print('Sampling freqency of the signal: ' + str(mysig_PN.fs) + ' (Hz)')
print('Combined linewidth of the local oscillators: ' + str(lwdth) + ' (Hz)')
print('Ratio : ' + str(lwdth/mysig_PN.fs))
print('Variance of the phase noise: ' + str(2*np.pi*lwdth/mysig_PN.fs))

### 2.2. Polarization Mode Dispersion

When a signal is transmitted with a given polarization, as fibers have imperfections and are succeptible to environmental stresses, the two orthogonal modes of the signal might be forced to travel at different velocities. This effect changes the polarization and has to be compensated. In ***Code 2.11***, change the PMD values and see the effect on the transmitted signal. Note that, when no PMD value is defined, PMD is not applied. To change the PMD both the differential group delay (first-order PMD), 'dgd', and the rotation angle to principle states of polarization, 'theta', must be defined.
Remark that below, both modes of the signal were represented. While in previous plots only the one of the modes was represented.

- <font color='darkblue'>Study on how changing the 'dgd', and 'theta' values affects the signal. Consider the effects on both modes.</font>

***Code 2.11:***

In [ ]:
dgd = 10**(-1)
theta = np.pi/4

# apply_PMD requires dual mode signal 
mysig_PMD = impairments.simulate_transmission(sigRes,theta = theta, dgd = dgd) 
plot_histogram(mysig_PMD,'dgd = '+str(dgd)+' & theta = '+str(theta/np.pi)+'pi')
plot_power_time([sigRes, mysig_PMD],['Original','New: ' + 'dgd = '+str(dgd)+' & theta = '+str(theta/np.pi)+'pi'],L = 50)
plot_power_frequency([sigRes, mysig_PMD],['Original','New: ' + 'dgd = '+str(dgd)+' & theta = '+str(theta/np.pi)+'pi'])

### 2.3. Carrier Frequency Offset

CFO occurs due to two reasons: frequency mismatch between the oscillators at the transmitter and receiver, and the Doppler effect. The frequency mismatch is the main contribution to CFO, it occurs due to manufacturing imperfections of the devices. Usually, the frequency of the device is not exactly the one specified by the manufacturer. In fact, the frequency of the device might even change over time due to temperature variations, pressure, among other factors. The Doppler effect occurs when the transmitter, the receiver, or any part of the channel is in some kind of movement.

Usually, the accuracy of local oscillators is defined in terms of ppm (part per million). A typical deviation considered for these devices is around $\pm 20$ ppm. This means that in one million seconds ($10^6$ s) a deviation of $20$ seconds is considered to occur. If our system is operating at $25$ GHz our frequency offset will be about $0.5$ MHz ($\frac{20}{10^6} \cdot 25 \times 10^9$).

In ***Code 2.12***, apply a CFO by changing the frequency offset, freq_off.

- <font color='darkblue'>Study this effect, and compare it with the phase noise effect.</font>

***Code 2.12:***

In [ ]:
freq_off = 10**4;

mysig_CFO = impairments.simulate_transmission(sigRes,freq_off = freq_off) 
plot_histogram(mysig_CFO,'freq_off = '+str(freq_off),mode = 0)
plot_power_time([sigRes, mysig_CFO],['Original','New: ' + 'freq_off = '+str(freq_off)],L = 50,mode = 0)
plot_power_frequency([sigRes, mysig_CFO],['Original','New: ' + 'freq_off = '+str(freq_off)],mode = 0)

### 2.4. Dispersion

Dispersion refers to a broadening of the light pulses that occurs during the propagation through the fiber. This effect occurs due to physical properties of the transmission medium. When the light beam suffers an excessive spreading, some bits will "overflow" their time slots and overlap with adjacent bits. This makes it hard for the receiver to properly distinguish between adjacent bits, increasing the BER.
 
Here we will consider the dispersion, D, given in s/m, and the length L, given in m, for a centre wavelength of $1550$ nm.

- <font color='darkblue'>In ***Code 2.13***, Change D between 0 and 1 and L between 0 and 10. What do you observe?</font>

***Code 2.13:***

In [ ]:
D = 0.001;
L = 1;
wl0 = 1550e-9;

mysig_D = impairments.add_dispersion(sigRes, D, L, wl0) #impairments.simulate_transmission(mysig,D) 
plot_histogram(mysig_D,'Dispersion = '+str(D),mode = 0)
plot_power_time([sigRes, mysig_D],['Original','New: ' + 'Dispersion = '+str(D)],L = 50,mode = 0)
plot_power_frequency([sigRes, mysig_D],['Original','New: ' + 'Dispersion = '+str(D)],mode = 0)

## 3. Coherent Receiver: Signal Analysis

Generating the signals to be used in this section.

***Code 2.14:***

In [ ]:
M = 64
seed = [?] # mecanographic number
nmodes = 1
fb = 25e9
N = 2**16
fs = fb*2
Ts = 1/fs
Tsymbol = 1/fb
beta = 0.01

sig = signals.SignalQAMGrayCoded(M, N, nmodes=nmodes, fb=fb, bitclass=signals.RandomBits, seed = seed)
sigRes = sig.resample(fs, Ts = Tsymbol, beta = beta, taps=4001, renormalise=True)

### 3.1. Signal-to-Noise Ratio

In ***Code 2.15***, we will now simulate the transmission of the signal generated in ***Code 2.14***, such that at the end of the transmission line it has a SNR of 10.

- <font color='darkblue'>Consider different values of SNR, and compare the constellation obtained with the original one. Additionally, compute the signal's SER, BER, EVM, and MI, for various values of SNR. What can you conclude from the results obtained? For the analysis, plot the SER, the BER, the EVM, and MI, as a function of the SNR.</font>

***Code 2.15*** plots the SER as a function of SNR using .cal\_ser(). To plot the BER, the EVM, and the MI use .cal\_ber(), .cal\_evm(), and .cal\_mi() instead, changing the code accordingly. 

***Code 2.15:***

In [ ]:
SNR = np.linspace(0,30,3);

mysig_SER = np.zeros((len(SNR),2))
for i,snr in enumerate(SNR):
    sig_trans = impairments.simulate_transmission(sig,snr = snr)
    mysig_SER[i,:] = sig_trans.cal_ser()
    plot_histogram(sig_trans,'Transmitted with SNR = ' + str(snr),mode=0)
    plot_power_time([sig, sig_trans],['Original','Transmitted with SNR = ' + str(snr)],L = 100,mode=0)
    plot_power_frequency([sig, sig_trans],['Original QPSK','Transmitted QPSK with SNR = ' + str(snr)],mode=0)
    
fig1 = figure(output_backend="webgl",plot_height=250, plot_width=350)
fig1.line(SNR,mysig_SER[:,0],line_width=2, alpha=0.7)
fig1.xaxis.axis_label = "SNR"
fig1.yaxis.axis_label = "SER"
show(fig1)

- <font color='darkblue'>  Modify ***Code 2.16*** to generate signals considering both M-PSK and M-QAM constellations with different values of M. Then simulate the transmission of the generated signals considering the same SNR. What can you conclude from the results obtained in terms of SER, BER, EVM, and MI? It might be useful, at this stage, to compute these parameters as a function of SNR and overlap the result for several modulation formats. </font>

The M-QAM signal can be generated considering: <br>
sigQAM = signals.SignalQAMGrayCoded(M, N, nmodes=1, fb=25e9, bitclass=signals.RandomBits, seed = seed), <br>
while the M-PSK signal can be generated considering: <br>
sigPSK = signals.SignalPSKGrayCoded(M, N, nmodes=1, fb=25e9, bitclass=signals.RandomBits, seed = seed). <br>
It might be useful to plot the results as a function of log_2(M) instead of M.

***Code 2.16:***

In [ ]:
snr = 4
M = 4;

mysig_SER_QAM = signals.SignalQAMGrayCoded(M, N, nmodes=1, fb=25e9, bitclass=signals.RandomBits, seed = seed)
sigQAM_trans = impairments.simulate_transmission(sigQAM,snr = snr)
mysig_SER_QAM = sigQAM_trans.cal_ser()
    
fig1 = figure(output_backend="webgl",plot_height=250, plot_width=300)
fig1.circle(M,mysig_SER_QAM[:],alpha=0.7,color='blue',legend_label='M-QAM', size=6)
fig1.line(M,mysig_SER_QAM[:],line_width=2, alpha=0.7,color='blue',legend_label='M-QAM')
fig1.xaxis.axis_label = "M"
fig1.yaxis.axis_label = "SER"
fig1.legend.location = "bottom_right"

show(fig1)

The results obtained before were obtained considering the signal before pulse shaping. 

- <font color='darkblue'>Using ***Code 2.17***, for the same SNR comment and compare the values of the SER, the BER, and the EVM of the data.</font>

***Code 2.17:***

In [ ]:
mysig_SER = sig.cal_ser()
mysig_BER = sig.cal_ber()
mysig_EVM = sig.cal_evm()

print("Symbol Error Rate: "+str(mysig_SER))
print("Bit Error Rate: "+str(mysig_BER))
print("Error Vector Magnitude: "+str(mysig_EVM))

mysig_SER = sigRes.cal_ser()
mysig_BER = sigRes.cal_ber()
mysig_EVM = sigRes.cal_evm()

print("Symbol Error Rate: "+str(mysig_SER))
print("Bit Error Rate: "+str(mysig_BER))
print("Error Vector Magnitude: "+str(mysig_EVM))

### 3.2 Equalization

In ***Code 2.18***, the transmission of a signal is simulated with certain impairments as phase noise, SNR, and PMD.

***Code 2.18:***

In [ ]:
M = 64
seed = [?] # mecanographic number
nmodes = 2
fb = 25e9
N = 2**16

sig = signals.SignalQAMGrayCoded(M, N, nmodes=nmodes, fb=fb, bitclass=signals.RandomBits, seed = seed)

snr = 15
sigNoise = impairments.simulate_transmission(sig,snr = snr)

print("Symbol Error Rate: "+str(sigNoise.cal_ser()))
print("Bit Error Rate: "+str(sigNoise.cal_ser()))

fs = fb*2
Tsymbol = 1/fb
beta = 0.1
sigRes = sigNoise.resample(fs, Ts = Tsymbol, beta = beta, taps=4001, renormalise=True)

mysig = impairments.simulate_transmission(sigRes,lwdth=100e3)

mysig = impairments.simulate_transmission(mysig,theta = np.pi/5.5, dgd = 40e-12) 

plot_histogram(mysig,'Signal before equalizing',mode=0)

Equalization is the process of, at the receiver, reverting the impairments that occurred during the transmission through the channel, as for example phase noise or dispersion.
Equalizers have to estimate the distortion and noise of the signal in order to enable its compensation. This equalizer uses a number of taps of 25, defined by 'Ntaps'. The step size for the equaliser is set by 'mu'. Test this equalizer using ***Code 2.19*** and analyse the error.

- <font color='darkblue'>What is meant by the taps of an equalizer?</font>

- <font color='darkblue'>What did this equalizer compensate for? Did the equalizer improve the BER and SER values (Compare the values of the SER and BER before and after equalization)?</font>

***Code 2.19:***

In [ ]:
Ntaps = 25
mu = 2e-4

# Apply equaliser with 25 taps and a stepsize of mu = 2*10^(-4)
sig_out, wxy, err = equalisation.dual_mode_equalisation(mysig, (mu,mu), Ntaps)
plot_histogram(sig_out,'Signal after equalizing',mode=0)

# Analyse the convergence of the error
fig1 = figure(title="Error", output_backend="webgl",plot_height=200, plot_width=350)
fig1.line(np.arange(err[0].shape[1]), abs(err[0][0]), color='red', alpha=1, legend="X")
fig1.line(np.arange(err[0].shape[1]), abs(err[0][1]), color='blue', alpha=1, legend="Y")
fig1.xaxis[0].axis_label = "symbol"
fig1.yaxis[0].axis_label = "error"

# Check the taps
fig2 = figure(title="Error", output_backend="webgl",plot_height=200, plot_width=350)
fig2.line(np.arange(wxy[0].shape[1]), wxy[0][0].real, color='red', alpha=1, legend="hxx")
fig2.line(np.arange(wxy[0].shape[1]), wxy[0][1].real, color='blue', alpha=1, legend="hxy")
fig2.xaxis[0].axis_label = "tap"
fig2.yaxis[0].axis_label = "weight"

show(row(fig1, fig2))

The next step is to eliminate phase noise. For QPSK, this is done using the function phaserec.viterbiviterbi(sig_out, av_len), where av_len is the block length of samples to average over, as done in ***Code 2.20***.
For a M-QAM constellation the phase recovery process is different, and is considered in ***Code 2.21***. In this case, a blind phase search is performed, as proposed in [3].

***Code 2.20:***

In [ ]:
# For QPSK

sig_out, ph = phaserec.viterbiviterbi(sig_out, 10)
sig_out = helpers.dump_edges(sig_out, 20)
sig_out = helpers.normalise_and_center(sig_out)
plot_histogram(sig_out,'Signal after equalizing',mode=0)

For a M-QAM constellation the phase recovery process is different, and is considered in ***Code 2.24***. Here a blind phase search is performed, as proposed in [3].

- <font color='darkblue'>After this equalization, looking at the results of ***Code 2.23*** and of ***Code 2.24***, what can you conclude? What tells you that the phase noise was compensated?</font>


***Code 2.21:***

In [ ]:
# For higher order M-QAM

sig_out, ph = phaserec.bps(sig_out, M, 40)
sig_out = helpers.normalise_and_center(sig_out)
plot_histogram(sig_out,'Signal after equalizing',mode=0)

### 3.2. Signal with pilots 

In optical communications it is useful to use pilot signals. These signals are often at different carrier frequencies than the transmitted signal. They are used in order to have a reference signal for synchronization, equalization, coherent detection, calibration, among other uses. The pilots are inserted into the data symbols periodically and removed at the receiver. Given that the phase of the pilots can be accurately estimated, they are used to estimate the phase of the data symbols. In this following section we will analyse a signal with pilot signals.

We will start by generating the original signal for the following work, using ***Code 2.22***. We will consider 64-QAM, with a overall frame length of $2^{16}$ bits, defined by the variable frame_len. This frame comprises the pilot sequence, the phase pilots, and the data payload. At the beginning of each frame we will have a sequence with $2^{10}$ pilots, defined by the variable pilot_seq_len. The number of pilots used is defined by Mpilots.

***Code 2.22:***

In [ ]:
M = 64 # QAM order: number of points in the constellation
frame_len = 2**16 # overall length of the signal comprised of the pilot sequence, the phase pilots and the data payload.
pilot_seq_len = 2**10 # number of pilots at the beginning of the frame
pilot_ins_rat = 32 # phase pilots are spaced every pilot_ins_symbol starting at the first symbol after the pilot sequence
nframes = 2 # how often to repeat the overall frame
pilot_scale = 1 # factor by which to multiply the pilots for power scaling
Mpilots = 4 # QAM order of the pilots in the sequence and the phase pilots
nmodes = 2 # number of spatial modes
fb = 24e9 # symbol rate of the signal

# Generate data with pilots 
mysig = signals.SignalWithPilots(M,frame_len,pilot_seq_len,pilot_ins_rat,
                                 nmodes=nmodes,Mpilots=Mpilots,nframes=nframes,fb=fb)
plot_constellation(mysig,'Original Signal With Pilots')

In ***Code 2.23***, we extract the pilots from the symbol using mysig.extract\_pilots(). To extract only the signal without pilots from the complete signal, one can use mysig.get\_data().

***Code 2.23:***

In [ ]:
# Obtain only the pilots from mysig, i.e., remove the data
mysig_only_pilots = mysig.extract_pilots()
plot_constellation(mysig_only_pilots,'Pilots')

# Obtain only the data from mysig, i.e., remore the pilots
mysig_without_pilots = mysig.get_data()
plot_constellation(mysig_without_pilots,'Original Signal')

In ***Code 2.24***, we apply pulse shaping to the signal with pilots.

***Code 2.24:***

In [ ]:
fs = fb*2
Tsymbol = 1/fb
beta = 0.1
sigRes = mysig.resample(fs, Ts = Tsymbol, beta = beta, taps=4001, renormalise=True)

plot_constellation(sigRes,'Original Signal With Pilots')

In ***Code 2.25***, the impairments of transmission are simulated.

***Code 2.25:***

In [ ]:
mysigTrans = impairments.simulate_transmission(mysig,snr=20,dgd=10e-12, freq_off=50e6,lwdth=100e3,roll_frame_sync=True)
plot_constellation(mysigTrans,'Original Signal With Pilots')

Finally, at the receiver, in ***Code 2.26***, the signal is synchronized, the frequency offset is corrected, and equalization is performed using the pilot signals.

- <font color='darkblue'>How can pilot signals be used for equalization? </font>

***Code 2.26:***

In [ ]:
# Synchronization
mysigTrans.sync2frame()

# Correct coarse frequency offset found by the syncronization algorithm
mysigTrans.corr_foe()

# Run the pilot-based Equalizer
wxy, eq_sig = equalisation.pilot_equaliser(mysigTrans,(1e-3,1e-3),45,foe_comp=False,methods=("cma","sbd_data"))

cpe_sig, ph = phaserec.pilot_cpe(eq_sig,N=5,use_seq=False)

#fig,axs = plt.subplots(ncols=2,nrows=2)

#axs[0,0].hist2d(eq_sig[0].real, eq_sig[0].imag, bins=200)
#axs[0,1].hist2d(eq_sig[1].real, eq_sig[1].imag, bins=200)
#axs[1,0].hist2d(cpe_sig[0].real, cpe_sig[0].imag, bins=200)
#axs[1,1].hist2d(cpe_sig[1].real, cpe_sig[1].imag, bins=200)

#fig.suptitle('Signal after phase recovery')

plt.figure()

plt.subplot(121)
plt.hist2d(eq_sig[0].real, eq_sig[0].imag, bins=200)
plt.subplot(122)
plt.hist2d(eq_sig[1].real, eq_sig[1].imag, bins=200)
plt.suptitle('Signal after equalization')
plt.show()

plt.figure()
plt.subplot(121)
plt.hist2d(cpe_sig[0].real, cpe_sig[0].imag, bins=200)
plt.subplot(122)
plt.hist2d(cpe_sig[1].real, cpe_sig[1].imag, bins=200)
plt.suptitle('Signal after phase recovery')
plt.show()

## References

[1] Michel C. Jeruchim, Philip Balaban, and K. Sam Shanmugan, ”Simulation of Communication Systems: Modeling, Methodology and Techniques”, Kluwer Academic Publishers, USA.

[2] Jochen Schr  ̈oder and Mikael Mazur, ”QAMPy a DSP chain for optical communications, DOI: 10.5281/zenodo.1195720”

[3] Timo Pfau et al, Hardware-Efficient Coherent Digital Receiver Concept With Feedforward Carrier Recovery for M-QAM Constellations, Journal of Lightwave Technology 27, pp 989-999 (2009).

[4] Kazovsky et al., Broadband Optical Access networks , Wiley Interscience , 2011.